In [1]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False)

/Users/sm/anaconda3/lib/python3.11/site-packages/sklearn/datasets/_openml.py:1002: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [2]:
mnist.keys()  # extra code – we only use data and target in this notebook

dict_keys(['data', 'target', 'frame', 'categories', 'feature_names', 'target_names', 'DESCR', 'details', 'url'])

In [3]:
X, y = mnist.data, mnist.target

X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [4]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

y_train_5 = (y_train == '5')  # True for all 5s, False for all other digits
y_test_5 = (y_test == '5')

In [5]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

SGDClassifier(random_state=42)

In [6]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')

array([0.95035, 0.96035, 0.9604 ])

In [7]:
from sklearn.dummy import DummyClassifier
dummy_clf =DummyClassifier()
dummy_clf.fit(X_train, y_train_5)

print(any(dummy_clf.predict(X_train)))

False


In [8]:
cross_val_score(dummy_clf, X_train, y_train_5, cv=3, scoring='accuracy')

array([0.90965, 0.90965, 0.90965])

Accuracy is genereally not a good performance measure for classificaiton. As we can see, only 10 % of images are 5,s so if you guessing that that an image is not a 5 you will have 90 % accuracy. (especially in skewed data)

Code down below:

clone(sgd_clf) creates a fresh clone of the classifier. This is done so that the original classifier (sgd_clf) is not affected by the training and can be used later.


In [9]:
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3)  # add shuffle=True if the dataset is not
                                       # already shuffled
for train_index, test_index in skfolds.split(X_train, y_train_5):
    clone_clf = clone(sgd_clf)
    X_train_folds = X_train[train_index]
    y_train_folds = y_train_5[train_index]
    X_test_fold = X_train[test_index]
    y_test_fold = y_train_5[test_index]

    clone_clf.fit(X_train_folds, y_train_folds)
    y_pred = clone_clf.predict(X_test_fold)
    n_correct = sum(y_pred == y_test_fold)
    print(n_correct / len(y_pred))


0.95035
0.96035
0.9604


Stratified sampling:

Key function: Stratified sampling is used to sample the best samples from a population. Population is divided into a group called strata (contains homegenous information) and randomly sampling data from each stratum

Step 1) Divide the data (population) into different stratas. Remember a strata is just a group.

step 2) Chose of the sampling should be propertionte or dispropertionate 

step 3) Randomly sampling from each strata: if properpatione => then each strata in the sampel is equal irrespective of each other. IF it is dispropertionate each sample amount the proportion of the population size of the stratum

![Stratified Sampling](https://media.geeksforgeeks.org/wp-content/uploads/20210905231338/Stratified.JPG)

## Confusion Matrix 
note: Remember a perfect confusion matrix will only have true positives (TP) and true negative (TN)

In [10]:
from sklearn.model_selection import cross_val_predict 

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

In [11]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_train_5, sgd_clf.predict(X_train))

cm

array([[52316,  2263],
       [  601,  4820]])

As we can see 
- [True - Negatives: 52136 are succefully classfied as non-5's] I [False - Positives: 2263 are unfusuccefuly calssified as 5's]     
- True - Positives: 4820 are succefully classified as 5's I [False - Negatives: 601 are unsuccefully calssified as non 5's]   



## NOTE:

### remember that if the preceision is high, there is a low recall because the model is very picky and therefor only predicts the positive class when it is very sure
- 83,7 % of the time it detects an image that represents 5
- but it only detects 65 % of the images that represents a 5

#### simply: when it detects an image that represents a 5 it does with a precision of 83,7 % but it only detect 65 % images that are 5's





## Accuracy score and recall COMBINED INTO F12

Accuarcy: TP/ TP + FP

Recall: TP / TP + FN

theese are necessary to combine into f1: Harmonic mean of values (harmonic mean gives bigger weights for lower values). As a result F1 only becomes high when accuracy and recall is high.



![Optional Text](https://media.graphcms.com/output=format:jpg/output=format:webp/resize=width:667,height:566/K8KFL3DRmKQtheUw0N3c)

In [12]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision_score(y_train_5, y_train_pred) 

0.8370879772350012

In [13]:
recall_score(y_train_5, y_train_pred) 

0.6511713705958311

In [14]:
f1_score(y_train_5, y_train_pred)


0.7325171197343846

The precision recall tradeoff simply implies that when precision increases the recall will decrease and vice versa. This is how SGDCClassifer makes classificaiton decisions